In [1]:
# To do , I am using 2 different computational graphs, so need 2 different sessions
#[ done ] 

# add model restore / save capability
# [ done ]

###############################################################

# code for training and performing inference for audio classification for 4 categories :
# 1. dogs howling
# 2. space shuttle launching / its engine
# 3. racing cars
# 4. b52 bomber







In [2]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from scipy.io import wavfile
import six
#import tensorflow as tf

import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim

# the vggist imports are necessary for using pretrained model on audioset , provided by Google

/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
num_classes=4 # 4 categories
model_path='data/models/1' # where you wanna save your model
data_base_directory='data/data/' # this is used to load the training data files
beta=0.03 # hyperparameter for l2 regularization

alphabet_dict={0:'A' , 1:'B' , 2:'C' , 3:'D' , 4:'E', 5:'F' , 6:'G' ,7 :'H' , 8:'I', 9:'J'}
# this is used for extracting class from the filename when training

In [4]:
# this is used to implement my own custom logic for each trainining example ( Tf dataset API)
def process_function(input):
  

  input=data_base_directory + str(input ,'utf-8') 

  for i in range(num_classes):
     if alphabet_dict[i] in input:
         y=int(i)
   
  return  input,y


In [5]:
logfilepath='data/dataset.txt' # dataset file , each line contains name of the audio ( class is automatically extracted from the name) 
# for instance myfileA.wav belongs to class 0

dataset = tf.data.TextLineDataset(logfilepath).shuffle(buffer_size=800) #.map(map_function)
# create a tf Dataset, from above file, shuffle the entries, to prevent class imbalance

dataset = dataset.map(
    lambda filename: tuple(tf.py_func(
        process_function, [filename],  [tf.string,tf.int64])))
# use a custom logic function defined above(process_function) for each training data

dataset=dataset.batch(1)
# batch size of 1

# In[25]:

# create iterator for navigation 
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()


# Let me verify if my dataset is indeed, what, I am expecting ..

with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(200):
      value = sess.run(next_element)
      print(value)


[b'data/data/14_19_01_A.wav-200.wav']
[b'data/data/14_25_10_C.wav-8.wav']
[b'data/data/14_28_54_D.wav-11.wav']
[b'data/data/14_25_10_C.wav-266.wav']
[b'data/data/14_25_10_C.wav-314.wav']
[b'data/data/14_25_10_C.wav-45.wav']
[b'data/data/14_19_01_A.wav-86.wav']
[b'data/data/14_24_15_B.wav-57.wav']
[b'data/data/14_24_15_B.wav-1.wav']
[b'data/data/14_23_13_B.wav-44.wav']
[b'data/data/14_19_01_A.wav-142.wav']
[b'data/data/14_25_10_C.wav-32.wav']
[b'data/data/14_19_01_A.wav-153.wav']
[b'data/data/14_19_01_A.wav-26.wav']
[b'data/data/14_25_10_C.wav-206.wav']
[b'data/data/14_25_10_C.wav-223.wav']
[b'data/data/14_28_19_D.wav-2.wav']
[b'data/data/14_19_01_A.wav-22.wav']
[b'data/data/14_19_01_A.wav-27.wav']
[b'data/data/14_24_15_B.wav-60.wav']
[b'data/data/14_19_01_A.wav-118.wav']
[b'data/data/14_25_10_C.wav-29.wav']
[b'data/data/14_25_10_C.wav-5.wav']
[b'data/data/14_19_01_A.wav-1.wav']
[b'data/data/14_25_10_C.wav-84.wav']
[b'data/data/14_19_01_A.wav-68.wav']
[b'data/data/14_19_01_A.wav-69.wav'

In [6]:
# Parameters
learning_rate = 0.003
num_steps = 500 # training steps
#batch_size = 2
display_step = 100 # this maybe used to print loss, every num_steps

# Network Parameters
n_hidden_1 = 100 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 128 # this is 128, as Googles output audio embedding is of this size


# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.int32, [None])


In [7]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, num_classes])),
    
    'out': tf.Variable(tf.random_normal([n_hidden_1, num_classes]))
}
biases = {
    
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
# Create model
# this might vary, according to size of data, problem complexity, for me this works well

def neural_net(x):
    # Hidden fully connected layer 
    layer_1 =  tf.add(tf.matmul(x, weights['h1']), biases['out'])
  
    return layer_1

In [9]:
# Construct model
Logits = neural_net(X)

#Logits = tf.Print(Logits, [Logits,tf.shape(Logits)], message="This is a: ")

# prediction will only be used at inference time, because softmax op is included in cross_entropy_loss
prediction = tf.nn.softmax(Logits)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=Logits))

# I don't wanna overfit, so I am defining l2 regularization loss at weights
reg1 = tf.nn.l2_loss(weights['out'] ) 
reg2= tf.nn.l2_loss(weights['h1'] )

# combined loss
loss = tf.reduce_mean(cost  + reg2 * beta)

train_op = optimizer.minimize(loss)




correct_pred = tf.equal(tf.argmax(Logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [10]:
# create first graph, one provided by google, in it's own session
g_1 = tf.Graph()
with g_1.as_default():
  

                
                
  

  # Sessions created in this scope will run operations from `g_1`.
                sess1 = tf.Session()
        
                   # Define the model in inference mode, load the checkpoint, and
                   # locate input and output tensors.
        
                vggish_slim.define_vggish_slim(training=False)
                vggish_slim.load_vggish_slim_checkpoint(sess1, 'vggish_model.ckpt')
                features_tensor = sess1.graph.get_tensor_by_name(
                vggish_params.INPUT_TENSOR_NAME)
                embedding_tensor = sess1.graph.get_tensor_by_name(
                vggish_params.OUTPUT_TENSOR_NAME)



INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [11]:
# Create session for my network, and start training

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(num_steps):
        avg_cost = 0.
        
          # Save model weights to disk
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path)
        
        # initialize iterator
        sess.run(iterator.initializer)
        while True:
            try:
                Next_element=sess.run(next_element)
                input_wav_file_name=str(Next_element[0][0],'utf-8')
                print(input_wav_file_name)
                
                examples_batch = vggish_input.wavfile_to_examples(input_wav_file_name)
                print(examples_batch.shape)

                # Prepare a postprocessor to munge the model embeddings.
                pproc = vggish_postprocess.Postprocessor('vggish_pca_params.npz')
                
               
                # Run inference and postprocessing on first session 
                [embedding_batch] = sess1.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
                print(embedding_batch.shape)
                postprocessed_batch = pproc.postprocess(embedding_batch)
                print(postprocessed_batch.shape)
                  
                # since each audio sample , is of size 2 seconds, and Googl'es model creates embeddings for about <= 1 sec
                # of audio, I batch these 2 embeddings of each audio file, into a single batch .
                    
                y_temp=np.append( Next_element[1], [ Next_element[1]  ]  )
                
                print(y_temp)

              # train single example on my own network
                _ , c,l = sess.run([train_op, cost, Logits], feed_dict={X: postprocessed_batch,Y: y_temp })
            
                 
                print(l.shape)
                print(c)
      
               
              
            except tf.errors.OutOfRangeError:
              break
   
        
                
    # Save model weights to disk again , since if you don't write , this it won't save for final epoch
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)   

Model saved in file: data/models/1
data/data/14_25_10_C.wav-81.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
398.64722
data/data/14_28_19_D.wav-20.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
1008.50696
data/data/14_25_10_C.wav-273.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_23_13_B.wav-54.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
1649.9114
data/data/14_25_10_C.wav-308.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
211.932
data/data/14_25_10_C.wav-296.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1289.8367
data/data/14_25_10_C.wav-283.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
529.30774
data/data/14_19_01_A.wav-165.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
3129.7148
data/data/14_23_13_B.wav-21.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
728.95544
data/data/14_25_10_C.wav-141.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1171.5193
data/data/14_23_13_B.wav-14.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
1428.279
data/data/14_25_10_C.wav-31.wav


(2, 128)
(2, 128)
[3 3]
(2, 4)
1037.4094
data/data/14_19_01_A.wav-182.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
821.7351
data/data/14_19_01_A.wav-98.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
977.84265
data/data/14_19_01_A.wav-113.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_25_10_C.wav-244.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
512.609
data/data/14_25_10_C.wav-78.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_17_54_A.wav-30.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_28_54_D.wav-11.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
1399.4375
data/data/14_25_10_C.wav-206.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
163.78607
data/data/14_25_10_C.wav-27.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_29_19_D.wav-125.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
29.079502
data/data/14_25_10_C.wav-135.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_19_01_A.wav-4.wav
(2, 96, 64)
(2, 128)

(2, 128)
(2, 128)
[1 1]
(2, 4)
1133.8057
data/data/14_25_10_C.wav-239.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1190.1787
data/data/14_19_01_A.wav-37.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
206.25934
data/data/14_17_54_A.wav-6.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_19_01_A.wav-101.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
1260.4111
data/data/14_29_19_D.wav-161.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
520.75793
data/data/14_19_01_A.wav-138.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
783.28125
data/data/14_25_10_C.wav-207.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
388.60406
data/data/14_19_01_A.wav-46.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
985.3009
data/data/14_29_19_D.wav-33.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
3.1407678
data/data/14_25_10_C.wav-188.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1397.6545
data/data/14_24_15_B.wav-49.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
858.2742
data/data/14_29_19_D.wav-65

(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_19_01_A.wav-34.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
129.349
data/data/14_29_19_D.wav-29.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
89.55432
data/data/14_25_10_C.wav-119.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1123.323
data/data/14_25_10_C.wav-117.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
748.5191
data/data/14_29_19_D.wav-1.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-262.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
278.70886
data/data/14_25_10_C.wav-85.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_25_10_C.wav-241.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_19_01_A.wav-92.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
845.7574
data/data/14_29_19_D.wav-7.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
1696.0454
data/data/14_25_10_C.wav-349.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1266.1013
data/data/14_25_10_C.wav-200.wav
(2, 96, 64)
(2, 12

(2, 128)
(2, 128)
[2 2]
(2, 4)
390.64093
data/data/14_29_19_D.wav-48.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_19_01_A.wav-118.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
657.6033
data/data/14_25_10_C.wav-245.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1563.2882
data/data/14_25_10_C.wav-46.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
1038.7178
data/data/14_25_10_C.wav-204.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
271.82727
data/data/14_25_10_C.wav-100.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_25_10_C.wav-280.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_19_01_A.wav-141.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
506.83655
data/data/14_25_10_C.wav-129.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_31_43_D.wav-21.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-327.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
364.61322
data/data/14_25_10_C.wav-34.wav
(2, 96, 64)
(2,

...

(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-118.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_19_01_A.wav-21.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
927.90063
data/data/14_28_54_D.wav-9.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_29_19_D.wav-42.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-138.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
5.960464e-08
data/data/14_25_10_C.wav-171.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_19_01_A.wav-31.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_19_01_A.wav-151.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_29_19_D.wav-34.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_29_19_D.wav-153.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_19_01_A.wav-194.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_29_19_D.wav-19.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0

(2, 128)
(2, 128)
[1 1]
(2, 4)
0.0
data/data/14_25_10_C.wav-248.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_28_19_D.wav-29.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-202.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_25_10_C.wav-40.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_29_19_D.wav-127.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_17_54_A.wav-17.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
543.20374
data/data/14_28_19_D.wav-35.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_19_01_A.wav-53.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_19_01_A.wav-63.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_24_15_B.wav-7.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
437.0968
data/data/14_19_01_A.wav-61.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_29_19_D.wav-74.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data

(2, 128)
(2, 128)
[2 2]
(2, 4)
898.1444
data/data/14_23_13_B.wav-49.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
0.0
data/data/14_19_01_A.wav-109.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_29_19_D.wav-17.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-301.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_29_19_D.wav-172.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_31_43_D.wav-17.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_29_19_D.wav-65.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
242.68341
data/data/14_29_19_D.wav-97.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
61.64038
data/data/14_17_54_A.wav-27.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_25_10_C.wav-223.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_29_19_D.wav-9.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-289.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0

(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_25_10_C.wav-266.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_29_19_D.wav-96.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-136.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_29_19_D.wav-50.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_29_19_D.wav-52.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_19_01_A.wav-116.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_25_10_C.wav-70.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_28_19_D.wav-11.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_24_15_B.wav-11.wav
(2, 96, 64)
(2, 128)
(2, 128)
[1 1]
(2, 4)
0.0
data/data/14_17_54_A.wav-0.wav
(2, 96, 64)
(2, 128)
(2, 128)
[0 0]
(2, 4)
0.0
data/data/14_29_19_D.wav-21.wav
(2, 96, 64)
(2, 128)
(2, 128)
[3 3]
(2, 4)
0.0
data/data/14_25_10_C.wav-74.wav
(2, 96, 64)
(2, 128)
(2, 128)
[2 2]
(2, 4)
0.0
data/data/14_19

KeyboardInterrupt: 

In [25]:
# code for inference, I am choosing a single fragment of video of b52 bomber downloaded from youtube
# for mode info, check readme file

with tf.Session() as sess:
                sess.run(init)

                # Restore model weights from previously saved model
                load_path = saver.restore(sess, model_path)
                print("Model restored from file: %s" % load_path)
                #sess.run(iterator.initializer)

            
                # pass inference file ( must be <=1 second)
                examples_batch = vggish_input.wavfile_to_examples('data/inference/fragments/out024.wav')
                print(examples_batch.shape)

                # Prepare a postprocessor to munge the model embeddings.
                pproc = vggish_postprocess.Postprocessor('vggish_pca_params.npz')
                
               
                # Run inference and postprocessing on first net
                [embedding_batch] = sess1.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
                print(embedding_batch.shape)
                postprocessed_batch = pproc.postprocess(embedding_batch)
                print(postprocessed_batch.shape)
    
              
              # Run prediction on second net
                p = sess.run([ prediction], feed_dict={X: postprocessed_batch })
            
                 
                print(p)
                #print(c)
                #print(pre)
      #          print(p)
               
               
              
          

INFO:tensorflow:Restoring parameters from data/models/1
Model restored from file: None
(1, 96, 64)
(1, 128)
(1, 128)
[array([[1., 0., 0., 0.]], dtype=float32)]
